In [ ]:
!source activate tensorflow

In [3]:
!mkdir checkpoint

In [2]:
!ls data/train2014/ | wc 

   2105    2105   67360


In [1]:
# make data smaller
import os
import random
from glob import glob

for filename in glob('data/train2014/*'):
    if random.random() > 0.025: # use only 2.5% of the data
        os.remove(filename)

In [ ]:
# learn art style: train model
# upload steampunk image
!time python style.py --style ../steampunk.jpg --checkpoint-dir checkpoint --epochs 2 --checkpoint-iterations 10 --batch-size 16
#!nvidia-smi # read "Disp.A Memory-Usage: and "GPU Memory Usage" for how much GPU RAM usage; "Volatile GPU-Util" grossly inaccurate
!pip install glances # glances is way better than nvidia-smi in terms of accuracy and UI
!pip install nvidia-ml-py3
!glances
# remove test-dir and other parameters for now

# use small batches if training pictures are bigger



# on 2.5% of pictures and 16 minibatch:
# real    9m59.879s
# user    9m5.264s
# sys     0m48.336s

In [ ]:
# original
!mkdir test_dir
!time python style.py --style ../cross_style.jpg --checkpoint-dir checkpoint --epochs 10 --checkpoint-iterations 100 --batch-size 16 --test ../seagull_on_sidewalk_small.jpg --test-dir test_dir

#### retry

## appears that it will regress to black; maybe GPU RAM constrained?
## also uses CPU by default, I changed it to be GPU, don't know if RAM is constrained. Have to make this argparse
## every iteration causes checkpoint save (GPU not running) and inference on test image, so use high iteration number

# real    56m18.865s
# user    50m23.584s
# sys     4m13.656s

In [ ]:
# content weight doubled
!mkdir test_dir_content_weight_modified/
!time python style.py --style ../cross_style.jpg --checkpoint-dir checkpoint --epochs 10 --checkpoint-iterations 100 --batch-size 16 --test ../seagull_on_sidewalk_small.jpg --test-dir test_dir_content_weight_modified/ --content-weight 1.5e1

## appears that it will regress to black; maybe GPU RAM constrained?
## also uses CPU by default, I changed it to be GPU, don't know if RAM is constrained. Have to make this argparse
## every iteration causes checkpoint save (GPU not running) and inference on test image, so use high iteration number

# runtime was just an extra 20 seconds

In [ ]:
# learning rate x 10
!mkdir test_dir_lr_modified_x10
!time python style.py --style ../cross_style.jpg --checkpoint-dir checkpoint --epochs 10 \
    --checkpoint-iterations 100 --batch-size 16 --test ../seagull_on_sidewalk_small.jpg \
    --test-dir test_dir_lr_modified_x10/ --learning-rate 1e-2 &> test_dir_lr_modified_x10/runtime_lr_x10.txt

## appears that it will regress to black (False); maybe GPU RAM constrained? (False)
## also uses CPU by default, I changed it to be GPU, don't know if RAM is constrained. Have to make this argparse
## every iteration causes checkpoint save (GPU not running) and inference on test image, so use high iteration number

# real    56m19.089s
# user    50m22.908s
# sys     4m17.932s

In [ ]:
# learning rate x 5
!mkdir test_dir_lr_modified_x5
!time python style.py --style ../cross_style.jpg --checkpoint-dir checkpoint --epochs 10 \
    --checkpoint-iterations 100 --batch-size 16 --test ../seagull_on_sidewalk_small.jpg \
    --test-dir test_dir_lr_modified --learning-rate 5e-3 &> test_dir_lr_modified_x5/runtime_lr_x5.txt
    
## appears that it will regress to black; maybe GPU RAM constrained?
## also uses CPU by default, I changed it to be GPU, don't know if RAM is constrained. Have to make this argparse
## every iteration causes checkpoint save (GPU not running) and inference on test image, so use high iteration number

# real    45m48.311s
# user    42m41.188s
# sys     3m3.264s

# Learning rate still too high
# estimate 6 seconds per checkpoint save and evaluate. ~23% runtime just on this step
# possible to isolate time cost by simply not evaluating and checkpointing

In [ ]:
# apply style on another picture, compare CPU vs GPU
# I don't think there are any other important parameters
!time python evaluate.py --checkpoint checkpoint --in-path examples/style/rain_princess.jpg --out-path ../output_image.jpg

# about the same between GPU (slightly faster) and CPU
# real    0m5.287s
# user    0m3.556s
# sys     0m2.068s

In [ ]:
# apply style to video
# upload video: play_dead.mp4
# use smaller minibatch if video very high resolution
!time python transform_video.py --in-path ../play_dead.mp4   --checkpoint checkpoint   --out-path ../output_video.mp4   --device /gpu:0   --batch-size 16

# GPU
# real    0m19.682s
# user    0m22.548s
# sys     0m6.408s

# CPU
# real    5m50.725s
# user    21m21.868s
# sys     1m17.100s

In [3]:
# add the first part to GPU to reduce runtime by 22 seconds

# Speed
## Try: more GPUs
## Done: bigger minibatch, less pictures

# Quality: figure out the parameters such that improvement plateaus
## Try: increase style weight, increase learning rate
## Done: add test folder, increase content weight makes fewer iterations to get original picture back

# Notes:
## notice with more iterations, then color of original image more visible; I assume that means the content loss is being reduced
## GPU RAM is full on this EC2 instance by loading up model. Thank goodness it fits into memory! For more powerful GPU, can probably increase minibatch for evaluation
## Currently GPU RAM constrained; not sure how it's evaluating testing picture without sufficient GPU RAM

In [ ]:
--style, --checkpoint-dir # explicitly required
--train-path, --vgg-path # implicitly required

--epochs, --checkpoint-iterations, --batch-size, # optional
--test, --test-dir, --content-weight, --style-weight, --tv-weight, --learning-rate # more optional

In [ ]:
real    763m46.949s
user    1192m59.980s
sys     74m2.732s

In [ ]:
# notes:
4 CPUs at full power vs 1 Tesla k80 (RAM: 12 GB; Compute Capability: 3.7)
714 second/iter vs 26 sec/iter   where iter is really 10 iters


p3 RAM 16 GB; Compute Capability 7.0 -- ~8 sec/iter (386 sec/iter on CPU cuz 8 CPUs)
bullion would have substantially more CPUs and more powerful GPU with more RAM, CC 6.0?
bullion has a premium GPU. Smaller GPUs like for gaming and bitcoin mining are not large enough to load in a GPU RAM.
Our model requires 9 GB of GPU RAM.

taskset -p CPU_id PID

the reason that things are faster is 2 parts: a) persistent GPU RAM which uses more than CPU RAM (which probably gets garbaged collected)
        b) lots more cores in GPU

In [ ]:
!wget https://s3.amazonaws.com/style-transfer-installation/Installing_TensorFlow_with_CUDA.sh
!time bash Installing_TensorFlow_with_CUDA.sh

# delete ffmpeg installation: conda install -y ffmpeg -c conda-forge